In [21]:
import numpy as np
import pandas as pd
import time

In [22]:
f = open("MPI/METIS/in-2004.graph")
meta = list(map(int, f.readline().split()))
e = meta[1]
n = meta[0]

G = {}
for i in range(1, n+1):
    G[i] = []
    edges = list(map(int, f.readline().split()))
    for nei in edges:
        if i != nei:
            G[i].append(nei)
            
f.close()

In [23]:
def densest(G, nodes, n, e):
    prev_core = None
    densities = []
    deg = {}
    bucket = []
    pos = {}
    vert = []
    edges = e
    me = e
    mv = n
    posmc = None
    mcore = 0
    mden = 0
    md = 0
    
    for v in nodes:
        deg[v] = 0
        vert.append(None)
        pos[v] = None
    
    #easily parallelizable
    for v in nodes:
        d = 0
        for u in G[v]:
            d += 1
        
        deg[v] = d
        if d > md:
            md = d
    
    for d in range(md + 1):
        bucket.append(0)
    
    for v in nodes:
        bucket[deg[v]] += 1
        
    start = 0
    for d in range(md + 1):
        num = bucket[d]
        bucket[d] = start
        start += num
    
    #easily parallalizable
    for v in nodes:
        pos[v] = bucket[deg[v]]
        vert[pos[v]] = v
        bucket[deg[v]] += 1
    
    #easily parallalizable
    for d in range(md, 0, -1):
        bucket[d] = bucket[d - 1]
    bucket[0] = 0
    
    for i in range(n):
        v = vert[i]
        core = deg[v]
        
        if prev_core != core:
            density = edges / (n - i)
            densities.append([core, density])
            if density > mden:
                mden = density
                mcore = core
                me = edges
                mv = n - i
            prev_core = core
        
        for u in G[v]:
            if pos[u] > pos[v]:
                edges -= 1
                
            if deg[u] > deg[v]:
                du = deg[u]
                pu = pos[u]
                pw = bucket[du]
                w = vert[pw]
                
                if u != w:
                    pos[u] = pw
                    vert[pu] = w
                    pos[w] = pu
                    vert[pw] = u
                    
                bucket[du] += 1
                deg[u] -= 1
    
    #print(mden, me, mv)
    posmc = bucket[mcore]
    for i in range(posmc - 1, -1, -1):
        v = vert[i]
        
        if deg[v] < mden:
            break
            
        legits = 0
        for u in G[v]:
            if pos[u] >= posmc:
                legits += 1
        
        #print(mden, deg[v], legits)
        if legits > mden:
            w = vert[posmc - 1]
            pos[v] = posmc - 1
            vert[i] = w
            pos[w] = i
            vert[posmc - 1] = v
            posmc -= 1
            me += legits
            mv += 1
            mden = me / mv
            
    return densities, vert, pos, posmc, me, mv, mden

In [24]:
start_time = time.clock()
densities, vert, pos, posmc, me, mv, mden = densest(G, range(1, n+1), n, e)
print(time.clock() - start_time, "seconds")

22.617630999999996 seconds


In [25]:
me, mv

(1984768, 7105)

In [26]:
mden

279.34806474313865